In [14]:
import pandas as pd

In [15]:
# df=pd.read_csv('data.csv')
# df.head(5)

In [16]:
%%writefile datautility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,data_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  data_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting datautility.py


Writing YAML

In [17]:
%%writefile data_config.yaml
file_type: csv
dataset_name: df
file_name: data
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - text
    - source
    - text_length
    - prompt_id
    - word_count

Overwriting data_config.yaml


In [18]:
# Read config file
import datautility as util
config_data = util.read_config_file("data_config.yaml")

In [19]:
config_data['inbound_delimiter']
config_data

{'file_type': 'csv',
 'dataset_name': 'df',
 'file_name': 'data',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['text', 'source', 'text_length', 'prompt_id', 'word_count']}

In [20]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

C:\Users\vasuv\AppData\Local\Temp\ipykernel_75652\270781102.py:5: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(source_file,config_data['inbound_delimiter'])


,text,source,prompt_id,text_length,word_count
0,"Federal law supersedes state law, and cannabis...",Bloom-7B,0,967,157
1,Miles feels restless after working all day. He...,Bloom-7B,0,5068,778
2,So first of I am danish. That means that I fol...,Bloom-7B,0,1602,267
3,In this paper we present a novel rule-based ap...,Bloom-7B,0,5469,848
4,"Most social progressives, love democracy, and ...",Bloom-7B,0,2379,380


In [21]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [22]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['text', 'source', 'prompt_id', 'text_length', 'word_count'], dtype='object')
columns of YAML are: ['text', 'source', 'text_length', 'prompt_id', 'word_count']


In [23]:
df.to_csv('output_file.csv.gz', sep='|', index=False, compression='gzip')

In [26]:
import os
import sys
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    sys.exit("File rejected due to validation failure")
else:
    print("col validation passed")
    
    num_rows = len(df)
    print(f'Total number of rows: {num_rows}')

    # Total number of columns
    num_columns = len(df.columns)
    print(f'Total number of columns: {num_columns}')

    # File size
    file_size = os.path.getsize('output_file.csv.gz') 
    print(f'File size: {file_size} bytes')

column name and column length validation passed
col validation passed
Total number of rows: 788922
Total number of columns: 5
File size: 842954452 bytes
